In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName ("trip_count_by_zone_sql").getOrCreate()
# spark instance 생성 

22/10/04 20:49:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
directory = "/Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data"
trip_file = "tripdata_2021_1-7.csv"
zone_file = "taxi+_zone_lookup.csv"

In [7]:
trip_data = spark.read.csv(f"file:///{directory}/{trip_file}", inferSchema = True, header = True)
zone_data = spark.read.csv(f"file:///{directory}/{zone_file}", inferSchema = True, header = True)
# 사전에 정의한 내용으로 csv파일을 읽어올 수 있음 
# inferSchema : 자동으로 스키마를 추론할건지 
# header : header 가지고 올 껀지 
# DataFrame이 생김 

In [7]:
trip_data.show(5)
# head()랑 비슷하다고 생각하면 됨 

22/10/04 20:31:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
 Schema: _c0, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
Expected: _c0 but found: 
CSV file: file:///Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data/tripdata_2021_1-7.csv
+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+--

In [8]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [9]:
# sql을 사용하기 위해 TempView 등록 
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")
# 해당 df를 SQL에서 사용하기 위해서 temporaryView에 등록 

In [10]:
spark.sql("select * from trip_data limit 5").show()

22/10/04 20:52:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
 Schema: _c0, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
Expected: _c0 but found: 
CSV file: file:///Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data/tripdata_2021_1-7.csv


+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|_c0|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|  0|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|            1.0|          2.1|       1.0|                 N|         142|          43|           2|        8.0|  3.0| 

In [11]:
spark.sql("select * from zone_data limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [16]:
# 맨하탄에서 승차하는 승객이 몇명이였는지 확인 
spark.sql("select borough, count(*) from (select zone_data.Borough as borough from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationId) group by borough").show()  
# 2개의 테이블을 조인한 결과로 borough별로 승차한 승객이 몇명이였는지 확인할 수 있음 

+-------------+--------+
|      borough|count(1)|
+-------------+--------+
|       Queens|  890742|
|          EWR|     793|
|      Unknown|  189197|
|     Brooklyn|  224576|
|Staten Island|    2195|
|    Manhattan|13597968|
|        Bronx|   95465|
+-------------+--------+



In [17]:
# 맨하탄에서 승차하는 승객이 몇명이였는지 확인 
spark.sql("select borough, count(*) from (select zone_data.Borough as borough from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationId) group by borough").show()  
# 2개의 테이블을 조인한 결과로 borough별로 하차한 승객이 몇명이였는지 확인할 수 있음 

+-------------+--------+
|      borough|count(1)|
+-------------+--------+
|       Queens|  665625|
|          EWR|   17660|
|      Unknown|  136029|
|     Brooklyn|  653582|
|Staten Island|    5891|
|    Manhattan|13302635|
|        Bronx|  219514|
+-------------+--------+

